In [1]:
import os

import pandas as pd
import numpy as np

from usa_spent.extract import extract
from usa_spent.transform import get_transaction_type
from usa_spent.transform import state_code

In [2]:
data_path = os.path.join(os.path.realpath('.'), 'data')

In [3]:
if not os.path.exists(data_path):
    os.mkdir(data_path)

In [4]:
file_loc = os.path.join(data_path, 'output_0301_02.csv')

# Extract

In [5]:
extract('2017-03-01', '2017-03-02', file_loc)

Searching params: {'filters': [{'field': 'action_date', 'operation': 'less_than_or_equal', 'value': '2017-03-02'}, {'field': 'action_date', 'operation': 'greater_than_or_equal', 'value': '2017-03-01'}]}
Completed data pull! 21327 records found


# Transform

In [8]:
df = pd.read_csv(file_loc, low_memory=False)

In [9]:
df = df[sorted(list(df.columns))].copy()

### Set transaction type as loan, grant, or contract

###### Loan is from "type_description" containing "loan". Grant is character in "type" field.  Contract is numeric in "type" field.

In [10]:
df['TRANSACTION_TYPE'] = df.apply(lambda row: get_transaction_type(row), axis=1)

### Set place of performance zip and state

In [12]:
df['POP_ZIP'] = df['place_of_performance__zip5']

###### Uses state code and state name if no state code. State name is then crosswalked back to state code

In [13]:
df['POP_STATE'] = df.apply(lambda row: state_code(row), axis=1)

### Set top tier funding & awarding agency

In [14]:
df['FUND_AGENCY'] = df['funding_agency__toptier_agency__name']
df['AWARD_AGENCY'] = df['awarding_agency__toptier_agency__name']

### Set sub tier funding & awarding agency

In [15]:
df['FUND_AGENCY_SUB'] = df['funding_agency__subtier_agency__name']
df['AWARD_AGENCY_SUB'] = df['awarding_agency__subtier_agency__name']

### Set product or service code

In [16]:
df['PSC'] = df['contract_data__product_or_service_code']

### Create NAICS field

###### NAICS number and NAICS description fields

In [17]:
df['NAICS'] = df['contract_data__naics']
df['NAICS_DESC'] = df['contract_data__naics_description']

### Create Date Fields

In [18]:
df['ACTION_DT'] = df['action_date'] 
df['START_DT'] = df['period_of_performance_start_date']
df['CURR_END_DT'] = df['period_of_performance_current_end_date']

### Obligation Dollar Amount

In [19]:
df['OBLIGATION_AMT'] = df['federal_action_obligation']

### Create output csv file

In [20]:
output_columns = ['TRANSACTION_TYPE','ACTION_DT','START_DT','CURR_END_DT','POP_ZIP','POP_STATE','OBLIGATION_AMT',
                 'PSC','NAICS','NAICS_DESC','FUND_AGENCY','AWARD_AGENCY','FUND_AGENCY_SUB','AWARD_AGENCY_SUB']

In [21]:
df[output_columns].to_csv(os.path.join(data_path, '0301 clean output.csv'), index = False)